In [1]:
import os
import sys
import random
import numpy as np
import tensorflow as tf
import xml.etree.ElementTree as ET
import datetime

In [2]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string(
    'dataset_name', 'pascalvoc',
    'The name of the dataset to convert.')
tf.app.flags.DEFINE_string(
    'dataset_dir', './data/VOCdevkit/VOC2007/',
    'Directory where the original dataset is stored.')

In [3]:
VOC_LABELS = {
    'none': (0, 'Background'),
    'aeroplane': (1, 'Vehicle'),
    'bicycle': (2, 'Vehicle'),
    'bird': (3, 'Animal'),
    'boat': (4, 'Vehicle'),
    'bottle': (5, 'Indoor'),
    'bus': (6, 'Vehicle'),
    'car': (7, 'Vehicle'),
    'cat': (8, 'Animal'),
    'chair': (9, 'Indoor'),
    'cow': (10, 'Animal'),
    'diningtable': (11, 'Indoor'),
    'dog': (12, 'Animal'),
    'horse': (13, 'Animal'),
    'motorbike': (14, 'Vehicle'),
    'person': (15, 'Person'),
    'pottedplant': (16, 'Indoor'),
    'sheep': (17, 'Animal'),
    'sofa': (18, 'Indoor'),
    'train': (19, 'Vehicle'),
    'tvmonitor': (20, 'Indoor'),
}

In [4]:
# Original dataset organisation.
DIRECTORY_ANNOTATIONS = 'Annotations/'
DIRECTORY_IMAGES = 'JPEGImages/'

In [5]:
def _process_image(directory, name):
    """Process a image and annotation file.
    Args:
      filename: string, path to an image file e.g., '/path/to/example.JPG'.
      coder: instance of ImageCoder to provide TensorFlow image coding utils.
    Returns:
      image_buffer: string, JPEG encoding of RGB image.
      height: integer, image height in pixels.
      width: integer, image width in pixels.
    """
    # Read the image file.
    filename = directory + DIRECTORY_IMAGES + name + '.jpg' #파일네임 
    image_data = tf.read_file(filename) #리드 
    print(type(image_data))
    print(image_data)
    print(image_data.shape)

    # Read the XML annotation file.
    filename = os.path.join(directory, DIRECTORY_ANNOTATIONS, name + '.xml') #엑스엠엘
    tree = ET.parse(filename) #파싱준비
    root = tree.getroot() #루트얻어옴 

    # Image shape.
    size = root.find('size')
    shape = [int(size.find('height').text),
             int(size.find('width').text),
             int(size.find('depth').text)]
    
    # Find annotations.
    bboxes = []
    labels = []
    labels_text = []
    difficult = []
    truncated = []
    for obj in root.findall('object'): #object 다 찾아서 넣음 바운딩박스/라벨(번호)/라벨텍스트(이름)/voc에 있는 특징? 어려운거?/좀짤린거
        label = obj.find('name').text
        labels.append(int(VOC_LABELS[label][0]))
        labels_text.append(label.encode('ascii'))

        if obj.find('difficult'):
            difficult.append(int(obj.find('difficult').text))
        else:
            difficult.append(0)
        if obj.find('truncated'):
            truncated.append(int(obj.find('truncated').text))
        else:
            truncated.append(0)

        bbox = obj.find('bndbox')
        bboxes.append((float(bbox.find('ymin').text) / shape[0],
                       float(bbox.find('xmin').text) / shape[1],
                       float(bbox.find('ymax').text) / shape[0],
                       float(bbox.find('xmax').text) / shape[1]
                       ))
    return image_data, shape, bboxes, labels, labels_text, difficult, truncated

In [7]:
path = os.path.join(FLAGS.dataset_dir, DIRECTORY_ANNOTATIONS)
filenames = sorted(os.listdir(path))
print(len(filenames))

5011


In [13]:
img_name = '000041'
_process_image(FLAGS.dataset_dir, img_name)

<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("ReadFile_1:0", shape=(), dtype=string)
()


(<tf.Tensor 'ReadFile_1:0' shape=() dtype=string>,
 [333, 500, 3],
 [(0.14114114114114115, 0.726, 0.3213213213213213, 0.864),
  (0.27627627627627627, 0.432, 0.9069069069069069, 0.614),
  (0.4444444444444444, 0.328, 0.7327327327327328, 0.454)],
 [20, 15, 15],
 [b'tvmonitor', b'person', b'person'],
 [0, 0, 0],
 [0, 0, 0])

In [20]:
class imdb(object):
    #image data base. 뭐를 넣을까.
    
    def __init__(self, name):
        self._name = name #이미지 이름
        self._classes = [] #각 클래스별 인덱스들
        self._rois = {} #각 클래스별 좌표들
        self._image_idx = [] #이미지 번호

    @property
    def name(self):
        return self._name
    
    @property
    def classes(self):
        return self._classes
    
    @property
    def num_classes(self):
        return len(self._classes)
    
    @property
    def rois(self):
        return self._rois
    
    @property
    def image_idx(self):
        return self._image_idx
    
    def read_from_directory(self, data_path=None):
        #data path 잡음
        #갯수 읽어옴
        #포문 돌면서 다 imdb에 읽어옴
        return

    def read_batch(self, shuffle=True):
        #읽어온 imdb를 배치해줌.
        return